In [ ]:
#client
import sys
import socket
import time
import threading
import random
import numpy as np
from tqdm import tqdm

#define
HOST="127.0.0.1"
SERVER_RECEIVE_PORT=50000 #服务端接收报文
SERVER_ACK_PORT=50001 #服务端发送ACK 非必须
CLIENT_SEND_PORT=50002 #客户端发送报文 非必须
CLIENT_ACK_PORT=50003 #客户端接收ACK
MAX_DATA_LENGTH=4194304 #最大报文长度4M
SSTHRESH=16 #拥塞窗口阈值
SWND=2 #初始发送窗口
TIMEOUT=2 #超时重传时间

def msg_sender(msg_array,ack_array):
    #信息发送套接字
    m_sender=socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    m_sender.bind((HOST,CLIENT_SEND_PORT))

    time_array=np.zeros_like(msg_array,dtype=float)
    segment_p=0
    swnd_p=0
    with tqdm(total=len(msg_array)) as pbar:
        while(True):
            if(segment_p==len(msg_array)):#发送完成
                pbar.update(SWND)
                m_sender.close()
                break
            while ack_array[swnd_p]:#移动窗口
                swnd_p+=1
                pbar.update()
            if segment_p==swnd_p+SWND:
                for _ in range(segment_p-SWND,segment_p):
                    if ack_array[_]==False and time.perf_counter()-time_array[_]>TIMEOUT:
                        m_sender.sendto(str(msg_array[_]).encode("utf8"),(HOST,SERVER_RECEIVE_PORT))
                continue
            m_sender.sendto(str(msg_array[segment_p]).encode("utf8"),(HOST,SERVER_RECEIVE_PORT))
            time_array[segment_p]=time.perf_counter()
            segment_p+=1


def ack_receiver(ack_array):
    #ack接收套接字
    ack_receiver=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    ack_receiver.bind((HOST,CLIENT_ACK_PORT))
    ack_receiver.settimeout(2*TIMEOUT)
    while(True):
        ack,addr=ack_receiver.recvfrom(1024)
        if ack:
            ack=int(ack.decode("utf8"))
            ack_array[ack]=True

class senderThread(threading.Thread):
    def __init__(self,msg_array,ack_array):
        threading.Thread.__init__(self)
        self.msg_array=msg_array
        self.ack_array=ack_array
    def run(self):
        msg_sender(self.msg_array,self.ack_array)

class receiverThread(threading.Thread):
    def __init__(self,ack_array):
        threading.Thread.__init__(self)
        self.ack_array=ack_array
    def run(self):
        ack_receiver(self.ack_array)


if __name__=="__main__":
    msg_length=128
    massage=np.zeros([msg_length],dtype=int)
    for _ in range(len(massage)):
        massage[_]=_
    ack=np.zeros([msg_length],dtype=bool)
    #print(massage.shape,ack.shape)
    thread1=senderThread(massage,ack)
    thread2=receiverThread(ack)
    thread1.start()
    thread2.start()

In [4]:
#server
import socket
import threading
import queue
import time
from random import random


#define
HOST="127.0.0.1"
SERVER_RECEIVE_PORT=50000 #服务端接收报文
SERVER_ACK_PORT=50001 #服务端发送ACK
CLIENT_SEND_PORT=50002 #客户端发送报文
CLIENT_ACK_PORT=50003 #客户端接收ACK
MAX_DATA_LENGTH=4194304 #最大报文长度4M
SWND=8 #发送窗口
TIMEOUT=2 #超时重传时间

def msg_receiver(rec_queue,p_drop):
    #接收发送端报文，存入rec_array，以概率p_drop丢包
    rec_socket=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    rec_socket.bind((HOST,SERVER_RECEIVE_PORT))
    rec_socket.settimeout(2*TIMEOUT)
    round_start=time.perf_counter()
    pkg_num=0
    while(True):
        round_end=time.perf_counter()
        if round_end-round_start>=1:
            print("speed:{}/s".format(pkg_num))
            pkg_num=0
            round_start=round_end
        msg,addr=rec_socket.recvfrom(1024)
        #print(msg.decode("utf8"),addr)
        msg=msg.decode("utf8")
        if msg:
            if random()>p_drop and not rec_queue.full():
                rec_queue.put(msg)
                pkg_num+=1
            continue

def ack_sender(rec_queue):
    #回复ack信息，线程间通过队列通信
    ack_socket=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    ack_socket.bind((HOST,SERVER_ACK_PORT))
    wc_dog=time.perf_counter()
    while(True):
        if not rec_queue.empty():
            ack=str(rec_queue.get()).encode("utf8")
            ack_socket.sendto(ack,(HOST,CLIENT_ACK_PORT))
            wc_dog=time.perf_counter()
            continue
        if time.perf_counter()-wc_dog>2*TIMEOUT:
            ack_socket.close()
            print("ACK TIMEOUT CLOSE")
            break


class msgrecThread(threading.Thread):
    def __init__(self,rec_queue,p_drop):
        threading.Thread.__init__(self)
        self.rec_queue=rec_queue
        self.p_drop=p_drop
    def run(self):
        msg_receiver(self.rec_queue,self.p_drop)

class acksendThread(threading.Thread):
    def __init__(self,rec_queue):
        threading.Thread.__init__(self)
        self.rec_queue=rec_queue
    def run(self):
        ack_sender(self.rec_queue)


if __name__=="__main__":
    msg_queue=queue.Queue(MAX_DATA_LENGTH)
    thread1=msgrecThread(msg_queue,0.03)
    thread2=acksendThread(msg_queue)
    thread1.start()
    thread2.start()

speed:6/s
speed:53/s
speed:106/s
speed:125/s
speed:198/s
speed:112/s
speed:149/s
speed:156/s
speed:160/s
speed:120/s
speed:144/s
speed:127/s
speed:124/s
speed:167/s
speed:145/s
speed:140/s
speed:135/s
speed:127/s
speed:136/s
speed:133/s
speed:127/s
speed:130/s
speed:123/s
speed:156/s
speed:107/s
speed:133/s
speed:137/s
speed:149/s
speed:120/s
speed:131/s
speed:116/s
speed:143/s


Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/xiongshihao/anaconda3/envs/test/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/tmp/ipykernel_3906895/3205212744.py", line 64, in run
  File "/tmp/ipykernel_3906895/3205212744.py", line 32, in msg_receiver
TimeoutError: timed out


ACK TIMEOUT CLOSE
